
# ComiCo: Federated Deep Learning Simulation

This notebook delivers the clean federated adaptation of the original ComiCo workflow. It follows the requested methodology: simulate heterogeneous network conditions, train local QoE models on each client, aggregate them federatively, and compare with centralized training.



## Context & Objectives

- Extend ComiCo to support multiple concurrent clients and distributed training.
- Simulate bandwidth dynamics for household, rural, and vehicular contexts (extensible to additional scenarios).
- Implement local learning loops and a central server that applies FedAvg-style aggregation (FedProx-ready).
- Analyse how update cadence, client count, and network instability influence convergence and robustness.
- Provide reusable utilities to scale from 3 up to 15 clients and drive comparative experiments versus the centralized baseline.



## 1. Initialisation

Import dependencies and configure deterministic behaviour for reproducible simulations.


In [1]:

import math
import random
from dataclasses import dataclass
from typing import Dict, List

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

DEVICE = torch.device("cpu")
plt.style.use("seaborn-v0_8-darkgrid")
pd.options.display.float_format = "{:,.3f}".format


ModuleNotFoundError: No module named 'torch'


## 2. Network & Playback Simulation

We encode bandwidth scenarios and utilities to synthesise per-client traces, including repeated playback loops that enrich local datasets.


In [2]:

@dataclass
class NetworkScenario:
    name: str
    base_throughput_kbps: float
    throughput_std: float
    playback_bitrate_kbps: float
    base_latency_ms: float
    latency_std: float
    base_loss: float
    loss_std: float
    initial_buffer_s: float
    rebuffer_threshold_s: float = 0.5


def simulate_network_trace(
    scenario: NetworkScenario,
    duration_s: int = 600,
    step_s: int = 1,
    seed: int | None = None,
) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    timesteps = np.arange(0, duration_s, step_s, dtype=float)

    throughput = rng.normal(
        loc=scenario.base_throughput_kbps,
        scale=scenario.throughput_std,
        size=timesteps.size,
    )
    throughput = np.clip(throughput, 200.0, None)

    latency = rng.normal(
        loc=scenario.base_latency_ms,
        scale=scenario.latency_std,
        size=timesteps.size,
    )
    latency = np.clip(latency, 5.0, None)

    loss = rng.normal(
        loc=scenario.base_loss,
        scale=scenario.loss_std,
        size=timesteps.size,
    )
    loss = np.clip(loss, 0.0, 1.0)

    buffer_levels: List[float] = []
    rebuffer_events: List[float] = []
    quality_ratio: List[float] = []
    qoe_scores: List[float] = []

    buffer_level = scenario.initial_buffer_s

    for sample_throughput, sample_latency, sample_loss in zip(throughput, latency, loss):
        download_time = scenario.playback_bitrate_kbps / max(sample_throughput, 1.0)
        buffer_delta = 1.0 - download_time
        buffer_level = max(buffer_level + buffer_delta, 0.0)

        rebuffer = float(buffer_level <= scenario.rebuffer_threshold_s)
        if rebuffer:
            buffer_level = scenario.initial_buffer_s * 0.6

        quality_ratio.append(float(np.clip(sample_throughput / scenario.playback_bitrate_kbps, 0.1, 3.0)))
        buffer_levels.append(buffer_level)
        rebuffer_events.append(rebuffer)

        penalty = (
            0.5 * (scenario.playback_bitrate_kbps / max(sample_throughput, scenario.playback_bitrate_kbps))
            + 0.3 * (sample_latency / (sample_latency + 40.0))
            + 0.2 * sample_loss
            + 0.35 * rebuffer
        )
        qoe_scores.append(float(np.clip(1.0 - penalty, 0.0, 1.0)))

    trace = pd.DataFrame(
        {
            "time_s": timesteps,
            "throughput_kbps": throughput,
            "latency_ms": latency,
            "loss_rate": loss,
            "buffer_level_s": buffer_levels,
            "rebuffer_event": rebuffer_events,
            "quality_ratio": quality_ratio,
            "qoe_score": qoe_scores,
            "scenario": scenario.name,
        }
    )
    return trace


def enrich_with_replays(
    trace: pd.DataFrame,
    repeats: int = 2,
    noise: float = 0.08,
    seed: int | None = None,
) -> pd.DataFrame:
    if repeats <= 0:
        return trace.copy()

    rng = np.random.default_rng(seed)
    augmented = [trace.copy()]
    for _ in range(repeats):
        jitter = rng.normal(loc=0.0, scale=noise, size=len(trace))
        perturbed = trace.copy()
        perturbed["throughput_kbps"] = np.clip(perturbed["throughput_kbps"] * (1 + jitter), 200.0, None)
        perturbed["latency_ms"] = np.clip(
            perturbed["latency_ms"] * (1 + rng.normal(0.0, noise / 2, len(trace))),
            5.0,
            None,
        )
        perturbed["loss_rate"] = np.clip(
            perturbed["loss_rate"] * (1 + rng.normal(0.0, noise, len(trace))),
            0.0,
            1.0,
        )
        perturbed["buffer_level_s"] = np.clip(
            perturbed["buffer_level_s"] * (1 + rng.normal(0.0, noise / 2, len(trace))),
            0.0,
            None,
        )
        perturbed["rebuffer_event"] = (perturbed["buffer_level_s"] <= 0.5).astype(float)
        augmented.append(perturbed)
    combined = pd.concat(augmented, ignore_index=True)
    combined.sort_values("time_s", inplace=True)
    combined.reset_index(drop=True, inplace=True)
    return combined


def describe_client_traces(traces: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    rows = []
    for client_id, df in traces.items():
        rows.append(
            {
                "client": client_id,
                "scenario": df["scenario"].iloc[0],
                "samples": len(df),
                "mean_throughput_kbps": df["throughput_kbps"].mean(),
                "mean_buffer_s": df["buffer_level_s"].mean(),
                "rebuffer_rate": df["rebuffer_event"].mean(),
                "mean_qoe": df["qoe_score"].mean(),
            }
        )
    return pd.DataFrame(rows).sort_values("client").reset_index(drop=True)



### Base Network Profiles

We instantiate representative bandwidth patterns (home, rural, vehicular). Additional clients reuse these templates with stochastic noise to reach 10-15 participants when required.


In [3]:

BASE_SCENARIOS = [
    NetworkScenario(
        name="home_stable",
        base_throughput_kbps=6500.0,
        throughput_std=450.0,
        playback_bitrate_kbps=4200.0,
        base_latency_ms=32.0,
        latency_std=6.0,
        base_loss=0.008,
        loss_std=0.003,
        initial_buffer_s=24.0,
    ),
    NetworkScenario(
        name="rural_unstable",
        base_throughput_kbps=2800.0,
        throughput_std=900.0,
        playback_bitrate_kbps=3600.0,
        base_latency_ms=85.0,
        latency_std=20.0,
        base_loss=0.035,
        loss_std=0.012,
        initial_buffer_s=18.0,
    ),
    NetworkScenario(
        name="vehicular_variable",
        base_throughput_kbps=4200.0,
        throughput_std=1400.0,
        playback_bitrate_kbps=3800.0,
        base_latency_ms=115.0,
        latency_std=28.0,
        base_loss=0.022,
        loss_std=0.009,
        initial_buffer_s=14.0,
    ),
]


def generate_client_traces(
    num_clients: int,
    duration_s: int = 600,
    base_seed: int = SEED,
    replay_factor: int = 2,
) -> Dict[str, pd.DataFrame]:
    rng = np.random.default_rng(base_seed)
    traces: Dict[str, pd.DataFrame] = {}
    for idx in range(num_clients):
        base = BASE_SCENARIOS[idx % len(BASE_SCENARIOS)]
        scenario_variant = NetworkScenario(
            name=f"{base.name}_v{idx + 1}",
            base_throughput_kbps=base.base_throughput_kbps * (1 + rng.normal(0.0, 0.08)),
            throughput_std=base.throughput_std * (1 + rng.normal(0.0, 0.15)),
            playback_bitrate_kbps=base.playback_bitrate_kbps * (1 + rng.normal(0.0, 0.05)),
            base_latency_ms=base.base_latency_ms * (1 + rng.normal(0.0, 0.12)),
            latency_std=max(5.0, base.latency_std * (1 + rng.normal(0.0, 0.2))),
            base_loss=float(np.clip(base.base_loss * (1 + rng.normal(0.0, 0.25)), 0.002, 0.2)),
            loss_std=float(np.clip(base.loss_std * (1 + rng.normal(0.0, 0.25)), 0.0005, 0.2)),
            initial_buffer_s=max(4.0, base.initial_buffer_s * (1 + rng.normal(0.0, 0.1))),
            rebuffer_threshold_s=base.rebuffer_threshold_s,
        )
        trace = simulate_network_trace(
            scenario_variant,
            duration_s=duration_s,
            seed=base_seed + 17 * (idx + 1),
        )
        if replay_factor:
            trace = enrich_with_replays(
                trace,
                repeats=replay_factor,
                noise=0.07,
                seed=base_seed + 31 * (idx + 1),
            )
        client_id = f"client_{idx + 1:02d}"
        trace["client"] = client_id
        traces[client_id] = trace.reset_index(drop=True)
    return traces


NameError: name 'SEED' is not defined

In [ ]:

CLIENT_COUNT = 3
client_traces = generate_client_traces(num_clients=CLIENT_COUNT, duration_s=600)
combined_traces = pd.concat(client_traces.values(), ignore_index=True)
client_summary = describe_client_traces(client_traces)
client_summary



### Scaling the Federation

Adjust CLIENT_COUNT (e.g. 10 or 15) in the cell above to extend the federation. All downstream steps automatically use the updated client dictionaries.



### Visual Diagnostics

Use the helper below to inspect how throughput, buffer level, latency, and QoE evolve per client.


In [ ]:

def plot_network_overview(
    traces: Dict[str, pd.DataFrame],
    metrics: List[str] | None = None,
    max_points: int | None = 400,
):
    metrics = metrics or ["throughput_kbps", "buffer_level_s", "latency_ms", "qoe_score"]
    num_clients = len(traces)
    num_metrics = len(metrics)
    fig, axes = plt.subplots(
        nrows=num_metrics,
        ncols=num_clients,
        figsize=(4 * num_clients, 3 * num_metrics),
        sharex="col",
    )
    if num_clients == 1:
        axes = np.array([[axes[row]] for row in range(num_metrics)])
    for col_idx, (client_id, df) in enumerate(sorted(traces.items())):
        subset = df if max_points is None else df.head(max_points)
        for row_idx, metric in enumerate(metrics):
            ax = axes[row_idx, col_idx]
            ax.plot(subset["time_s"], subset[metric])
            if row_idx == 0:
                ax.set_title(client_id)
            if col_idx == 0:
                ax.set_ylabel(metric)
    fig.tight_layout()
    return fig


In [ ]:

# Uncomment to visualise the first 400 samples for each client
# plot_network_overview(client_traces, max_points=400);



## 3. Data Preparation for Learning

We split each client dataset into train/test partitions and prepare tensors for PyTorch.


In [ ]:

FEATURE_COLUMNS = [
    "throughput_kbps",
    "buffer_level_s",
    "latency_ms",
    "loss_rate",
    "quality_ratio",
    "rebuffer_event",
]
TARGET_COLUMN = "qoe_score"

client_datasets: Dict[str, Dict[str, np.ndarray]] = {}
central_train_features = []
central_train_targets = []
central_test_features = []
central_test_targets = []

for client_id, frame in client_traces.items():
    X = frame[FEATURE_COLUMNS]
    y = frame[TARGET_COLUMN]
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        random_state=SEED,
        shuffle=True,
    )
    client_datasets[client_id] = {
        "X_train": X_train.to_numpy(dtype=np.float32),
        "X_test": X_test.to_numpy(dtype=np.float32),
        "y_train": y_train.to_numpy(dtype=np.float32),
        "y_test": y_test.to_numpy(dtype=np.float32),
    }
    central_train_features.append(X_train.to_numpy(dtype=np.float32))
    central_train_targets.append(y_train.to_numpy(dtype=np.float32))
    central_test_features.append(X_test.to_numpy(dtype=np.float32))
    central_test_targets.append(y_test.to_numpy(dtype=np.float32))

central_data = {
    "X_train": np.vstack(central_train_features),
    "y_train": np.concatenate(central_train_targets),
    "X_test": np.vstack(central_test_features),
    "y_test": np.concatenate(central_test_targets),
}



## 4. QoE Modelling Utilities

The helpers below are shared by the centralized and federated pipelines.


In [ ]:

def make_loader(features: np.ndarray, targets: np.ndarray, batch_size: int, shuffle: bool) -> DataLoader:
    features_tensor = torch.from_numpy(features).float()
    targets_tensor = torch.from_numpy(targets).float().unsqueeze(1)
    dataset = TensorDataset(features_tensor, targets_tensor)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)


class QoEPredictor(nn.Module):
    def __init__(self, input_dim: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

    def forward(self, features: torch.Tensor) -> torch.Tensor:
        return self.net(features)


def train_one_epoch(
    model: nn.Module,
    loader: DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: nn.Module,
) -> float:
    model.train()
    running_loss = 0.0
    total_samples = 0
    for features, target in loader:
        features = features.to(DEVICE)
        target = target.to(DEVICE)
        optimizer.zero_grad()
        predictions = model(features)
        loss = criterion(predictions, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * features.size(0)
        total_samples += features.size(0)
    return running_loss / max(total_samples, 1)


def evaluate_model(model: nn.Module, loader: DataLoader, criterion: nn.Module) -> Dict[str, float]:
    model.eval()
    predictions: List[np.ndarray] = []
    targets: List[np.ndarray] = []
    with torch.no_grad():
        for features, target in loader:
            features = features.to(DEVICE)
            target = target.to(DEVICE)
            outputs = model(features)
            predictions.append(outputs.cpu().numpy())
            targets.append(target.cpu().numpy())
    preds = np.vstack(predictions)
    trues = np.vstack(targets)
    loss_value = criterion(torch.from_numpy(preds), torch.from_numpy(trues)).item()
    mae_value = mean_absolute_error(trues, preds)
    rmse_value = math.sqrt(mean_squared_error(trues, preds))
    return {"loss": loss_value, "mae": mae_value, "rmse": rmse_value}


def evaluate_clients(model: nn.Module, splits: Dict[str, Dict[str, np.ndarray]], batch_size: int = 256) -> pd.DataFrame:
    criterion = nn.MSELoss()
    rows = []
    for client_id, split in sorted(splits.items()):
        loader = make_loader(split["X_test"], split["y_test"], batch_size=batch_size, shuffle=False)
        metrics = evaluate_model(model, loader, criterion)
        metrics["client"] = client_id
        rows.append(metrics)
    return pd.DataFrame(rows)



### 4.1 Centralized Baseline


In [ ]:

def train_centralized(
    data: Dict[str, np.ndarray],
    epochs: int = 30,
    batch_size: int = 128,
    lr: float = 1e-3,
) -> tuple[nn.Module, pd.DataFrame]:
    model = QoEPredictor(input_dim=data["X_train"].shape[1]).to(DEVICE)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    train_loader = make_loader(data["X_train"], data["y_train"], batch_size=batch_size, shuffle=True)
    val_loader = make_loader(data["X_test"], data["y_test"], batch_size=batch_size, shuffle=False)

    history: List[Dict[str, float]] = []
    for epoch in range(1, epochs + 1):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
        eval_metrics = evaluate_model(model, val_loader, criterion)
        eval_metrics.update({"epoch": epoch, "train_loss": train_loss})
        history.append(eval_metrics)
    return model, pd.DataFrame(history)



### 4.2 Federated Training (FedAvg)


In [ ]:

def federated_train(
    splits: Dict[str, Dict[str, np.ndarray]],
    rounds: int = 20,
    local_epochs: int = 2,
    batch_size: int = 64,
    lr: float = 5e-4,
) -> tuple[nn.Module, pd.DataFrame]:
    first_client = next(iter(splits.values()))
    model = QoEPredictor(input_dim=first_client["X_train"].shape[1]).to(DEVICE)
    criterion = nn.MSELoss()

    global_test_features = np.vstack([split["X_test"] for split in splits.values()])
    global_test_targets = np.concatenate([split["y_test"] for split in splits.values()])
    global_test_loader = make_loader(global_test_features, global_test_targets, batch_size=256, shuffle=False)

    history: List[Dict[str, float]] = []

    for round_idx in range(1, rounds + 1):
        aggregated_state = {key: torch.zeros_like(val) for key, val in model.state_dict().items()}
        total_samples = 0
        local_states = []
        local_weights = []

        for client_id, split in sorted(splits.items()):
            local_model = QoEPredictor(input_dim=first_client["X_train"].shape[1]).to(DEVICE)
            local_model.load_state_dict(model.state_dict())
            optimizer = torch.optim.Adam(local_model.parameters(), lr=lr)
            train_loader = make_loader(split["X_train"], split["y_train"], batch_size=batch_size, shuffle=True)

            for _ in range(local_epochs):
                train_one_epoch(local_model, train_loader, optimizer, criterion)

            local_state = {key: value.detach().clone() for key, value in local_model.state_dict().items()}
            sample_count = len(split["X_train"])
            local_states.append(local_state)
            local_weights.append(sample_count)
            total_samples += sample_count

        for state, weight in zip(local_states, local_weights):
            coeff = weight / total_samples
            for key in aggregated_state:
                aggregated_state[key] = aggregated_state[key] + state[key] * coeff

        model.load_state_dict(aggregated_state)
        metrics = evaluate_model(model, global_test_loader, criterion)
        metrics.update({"round": round_idx})
        history.append(metrics)

    return model, pd.DataFrame(history)



### 4.3 Unified Experiment Driver


In [ ]:

def run_experiment(
    client_splits: Dict[str, Dict[str, np.ndarray]],
    central_data: Dict[str, np.ndarray],
    central_epochs: int = 30,
    central_lr: float = 1e-3,
    central_batch: int = 128,
    fed_rounds: int = 25,
    fed_local_epochs: int = 2,
    fed_lr: float = 5e-4,
    fed_batch: int = 64,
) -> Dict[str, object]:
    central_model, central_history = train_centralized(
        central_data,
        epochs=central_epochs,
        batch_size=central_batch,
        lr=central_lr,
    )
    fed_model, fed_history = federated_train(
        client_splits,
        rounds=fed_rounds,
        local_epochs=fed_local_epochs,
        batch_size=fed_batch,
        lr=fed_lr,
    )
    central_eval = evaluate_clients(central_model, client_splits)
    fed_eval = evaluate_clients(fed_model, client_splits)

    comparison = central_eval.merge(fed_eval, on="client", suffixes=("_central", "_federated"))

    return {
        "central_model": central_model,
        "central_history": central_history,
        "federated_model": fed_model,
        "federated_history": fed_history,
        "comparison": comparison,
    }



## 5. Experimentation & Analysis

The snippets below illustrate how to launch baseline comparisons and sensitivity studies.


In [ ]:

# Example: baseline comparison for the current CLIENT_COUNT
# results = run_experiment(
#     client_splits=client_datasets,
#     central_data=central_data,
#     central_epochs=25,
#     fed_rounds=20,
#     fed_local_epochs=2,
# )
# display(results["comparison"])
# ax = results["federated_history"].plot(x="round", y=["loss", "mae", "rmse"], marker="o");
# ax.set_title("Federated metrics per round");
# ax.set_ylabel("Score");


In [ ]:

def compare_federated_configs(
    client_splits: Dict[str, Dict[str, np.ndarray]],
    configs: List[Dict[str, int | float | str]],
) -> pd.DataFrame:
    records = []
    for cfg in configs:
        label = cfg.get("label") or f"rounds={cfg['rounds']}, local_epochs={cfg['local_epochs']}"
        _, history = federated_train(
            client_splits,
            rounds=int(cfg.get("rounds", 20)),
            local_epochs=int(cfg.get("local_epochs", 2)),
            batch_size=int(cfg.get("batch_size", 64)),
            lr=float(cfg.get("lr", 5e-4)),
        )
        last_metrics = history.iloc[-1].to_dict()
        last_metrics["configuration"] = label
        records.append(last_metrics)
    return pd.DataFrame(records)


In [ ]:

# Example: study the impact of update frequency
# configs = [
#     {"label": "Fast updates", "rounds": 15, "local_epochs": 1},
#     {"label": "Balanced", "rounds": 20, "local_epochs": 2},
#     {"label": "Delayed", "rounds": 10, "local_epochs": 4},
# ]
# compare_federated_configs(client_datasets, configs)



### Next Steps

- Increase CLIENT_COUNT to 10 or 15 and rerun the workflow to assess scalability.
- Try alternative aggregation rules (e.g. FedProx) by adjusting 
ederated_train.
- Log additional QoS indicators (stall duration, bitrate switches) to enrich the QoE target.
- Persist experiment artefacts (histories, comparisons) to CSV for offline analysis when needed.
